In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
### Hyper paramas:

#dataset
IMG_SIZE = 128
BATCH_SIZE = 32 # maybe it bigger if it cans

#seeds
torch.manual_seed(36)
torch.cuda.manual_seed(36)


In [110]:
transform_data = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

data_ds = torchvision.datasets.OxfordIIITPet(root = "data" , split = "trainval",target_types = "category",download = True,transform=transform_data)


data_dl = DataLoader(data_ds,batch_size=BATCH_SIZE,shuffle=True)

In [111]:
len(data_ds.classes)

37

In [17]:
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv')!= -1:
    nn.init.normal_(m.weight.data,0.0,0.02)
  elif classname.find('BatchNorm')!= -1:
    nn.init.normal_(m.weight.data, 0.0,0.02)
    nn.init.constant_(m.bias.data,0)

In [45]:
### Discriminator :

def d_block(in_ch,out_ch,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.Conv2d(in_ch,out_ch,kernel,stride,padding,bias=False),
      nn.LeakyReLU(0.2,inplace=True),
      nn.BatchNorm2d(out_ch)
  )
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.embed_size = 32
    self.num_classes = 37
    self.embed = nn.Embedding(self.num_classes,self.embed_size,dtype=torch.float32 )

    self.model = nn.Sequential(
        ## bs x 3 x128 x128
        d_block(3,32), # -> 32 x 128 x 128
        d_block(32,64), # -> 64 x 64 x 64
        d_block(64,128), # -> 128 x 32 x 32
        d_block(128,256), # -> 256 x 16 x 16
        d_block(256,512), # -> 512 x 8 x 8
        d_block(512,32), # -> 32 x 4 x4
        nn.Flatten(),
    )
    self.model_2 = nn.Sequential(
        nn.Linear(160,100),
        nn.LeakyReLU(100,1),
        nn.Sigmoid(),
    )
    self.apply(weights_init)

  def forward(self,input,labels):
    x = self.model(input)
    labels = labels.to(torch.long)
    y = self.embed(labels)
    input = torch.cat([x,y],1)
    final_output = self.model_2(input)

    return final_output

In [ ]:
discriminator_model = Discriminator().to(device)
discriminator_model

In [ ]:
discriminator_model

In [124]:
def g_block(in_ch,out_ch,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.ConvTranspose2d(in_ch,out_ch,kernel,stride,padding,bias=False),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(out_ch),
  )

class Generator (nn.Module):
  def __init__(self):
    super().__init__()

    self.embed_size = 32
    self.num_classes = 37
    self.embed = nn.Embedding(self.num_classes,self.embed_size,dtype=torch.float32 )
    self.linear= nn.Linear(1,10)
    self.model = nn.Sequential(
        g_block(100+self.embed_size,512,stride=1,padding=0),
        g_block(512,256),
        g_block(256,128),
        g_block(128,64),
        g_block(64,32),
        g_block(32,3),
    )

    self.apply(weights_init)

  def forward(self,input_noise,labels):
    labels = self.linear(labels)
    labels = torch.tensor(labels, dtype=torch.long)
    label_embeddings = self.embed(labels).view(len(labels), self.embed_size, 1, 1),
    input = torch.cat([input_noise,label_embeddings],1)

    return self.model(input)

In [125]:
generator_model = Generator()
generator_model

Generator(
  (embed): Embedding(37, 32)
  (linear): Linear(in_features=1, out_features=10, bias=True)
  (model): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(132, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=

In [119]:
def noise(size):
  n = torch.randn(size,100,1,1,device=device)
  return n.to(device)

In [ ]:
def test_generator():
    # Create an instance of the Generator
    generator = Generator()

    # Generate fake noise and labels (batch size 1)
    input_noise = torch.randn(1, 100)
    labels = torch.randint(0, 37, (1,), dtype=torch.long)

    # Generate output from the Generator
    output = generator(input_noise, labels)

    # Print the output shape
    print("Output shape:", output.shape)

# Run the test
test_generator()

In [82]:
noise(100).shape

torch.Size([100, 100, 1, 1])

In [100]:
classes = data_ds.class_to_idx
torch.tensor(np.array(classes["Boxer"])).long()

TypeError: len() of a 0-d tensor

In [126]:
x,y = data_ds[1]
generator_model(x,y)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not int

In [ ]:
data_ds.transform

In [ ]:
t = transforms.ToTensor()
t(x)